# BLANK FEDS PEC (Polygon Evaluation and Comparison) Outline

### Summary

This notebook is a blank starting point for users to use the PEC implementation. Refer to `KINCADE_DEMO_FEDS_OUTLINE.ipynb` for a demonstration on how to use the outline

### User Inputs for Comparison: time, bbox, FEDS set, reference set

Note for inputs:

- FEDS Input: you must specify a valid title (e.g. firenrt) and collection (e.g. public.eis_fire_lf_perimeter_archive) [What is a veda input? Tell the user this before telling them to include a valid one]
- Reference Input: as above you must specify a valid title and access method such that the proper methods are applied [again, what is a reference input? Also more helpful to list the valid access methods.]

In [ ]:
import Utilities
import Input_FEDS
import Input_Reference
import Output_Calculation

from Input_FEDS import InputFEDS
from Input_Reference import InputReference
from Output_Calculation import OutputCalculation
from Utilities import *

In [ ]:
# this is the ONLY thing users are intended to modify!
# inputs for searching FEDS and nifc sets

# START TIME
year_start = ... # e.g. 2019
month_start = ... # e.g. 10
day_start = ... # e.g. 23
hour_start = ... # e.g. 0
minute_start = ... # e.g. 0
second_start = ... # e.g. 0
tz_offset_hours_start = ... # e.g. 0
tz_offset_minutes_start = ... # e.g. 0
utc_offset_start = ... # e.g. '00:00'

# END TIME
year_stop = ... # e.g. 2019
month_stop = ... # e.g. 11
day_stop = ... # e.g. 14
hour_stop = ... # e.g. 0
minute_stop = ... # e.g. 0
second_stop = ... # e.g. 0
tz_offset_hours_stop = ... # e.g. 0
tz_offset_minutes_stop = ... # e.g. 0
utc_offset_stop = ... # e.g. '00:00'

# CRS - 4326 RECOMMENDED FEDS FIRE PERIM DEFAULT
crs = ... # e.g. 3857

# BBOX FOR SEARCH - [lon, lat, lon, lat]
search_bbox = ... # e.g.  ["-122.887312", "38.560242", "-122.601345", "38.813395"] # kincade fire, 38.560242, -122.887312 left 38.813395, -122.601345 right


# FEDS INPUT SETTINGS [Change to FEDS Input settings]
feds_title = ... # e.g. "firenrt"
feds_collection =  ... # e.g. "public.eis_fire_lf_perimeter_archive"
feds_access_type = ... # e.g. "api" # or "local
feds_limit = ... # e.g. 1000 # amount of features to consider for FEDS API access; warning appears if it misses any entries
feds_filter = ... # e.g. False # False or a valid query: e.g. "farea>5 AND duration>2"
feds_apply_finalfire = ... # e.g. True # set this to true if you want the only the latest fireID to be taken per unique FireID

# REFERENCE INPUT SETTINGS 
ref_title =... # e.g.  "nifc_interagency_history_local" # this is a predefined set example
ref_control_type = ... # e.g. "defined" # or "custom"
ref_custom_url = ... # e.g. "none" # not custom for demo
ref_custom_read_type = ... # e.g. "none"  # not custom for demo
ref_filter = ... # e.g. False # False or a valid query

# OUTPUT SETTINGS
maap_username = ... # e.g. "ksharonin" # per user, in order to store into user's own public bucket
name_for_output_file = ... # e.g. f"firenrt_vs_nifc_interagency_{year_start}_{search_bbox[0]}_{search_bbox[1]}_{search_bbox[2]}_{search_bbox[3]}" # name for your file EXCLUDING PREFIX!
output_format = ... # e.g. "txt" # prefix txt, json, others coming
print_on = ... # e.g. True # print to jupyter nb
plot_on = ... # e.g. False # display plot of comparison
day_search_range = ... # e.g. 7 # acceptable distance to search from feds -> reference (e.g. if refernce polygon is 8 days away, it is not included in calculations)

# NOTE: for now don't modify this, just showing you how url is contructed
output_maap_url = ... # e.g. f"s3://maap-ops-workspace/shared/{maap_username}/{name_for_output_file}.{output_format}" # maap url to save; MUST BE MAAPS URL! (for now at least to make things simple

### Argument Processing: format and check to report any issues
- Note: upon instance instantiation we can also get reports back of invalidity -> any warnings/errors should be addressed

In [ ]:

# start date formatting
search_start = Utilities.format_datetime(year_start, 
                                         month_start, 
                                         day_start, 
                                         hour_start, 
                                         minute_start, 
                                         second_start, 
                                         tz_offset_hours_start, 
                                         tz_offset_minutes_start,
                                         utc_offset_start)
# stop date formatting
search_stop = Utilities.format_datetime(year_stop, 
                                        month_stop, 
                                        day_stop, 
                                        hour_stop, 
                                        minute_stop, 
                                        second_stop, 
                                        tz_offset_hours_stop, 
                                        tz_offset_minutes_stop,
                                        utc_offset_stop)

# bound check the bbox
assert Utilities.check_bbox(search_bbox), f"ERR: passed bbox {search_bbox} is not valid; check bounds"
assert  Utilities.check_crs(crs), f"ERR: invalid crs provided {crs}; please enter valid ESPG CRS number"

### Using Inputs: Instantiate Inputs + Calculation Output

With our inputs, we are ready to start; run this to get the program to fetch the FEDS candidates in the provided time/location window, along with an intersecting NIFC in the specific day range

In this case, we get the latest object of FEDS in the kincade zone + its NIFC closest match, which is about 6 days away and heavily intersecting

In [ ]:
# create instances to track properties
print('You may see an ERROR 1 occur; ignore this unless block throws actual exception...')

feds_firenrt = InputFEDS(
                 feds_title, 
                 feds_collection, 
                 search_start,
                 search_stop,
                 search_bbox,
                 crs,
                 feds_access_type,
                 feds_limit,
                 feds_filter,
                 feds_apply_finalfire
                )

nifc_search = InputReference( 
                 search_start,
                 search_stop,
                 search_bbox,
                 crs,
                 ref_title,
                 ref_control_type,
                 ref_custom_url,
                 ref_custom_read_type,
                 ref_filter,
                )

my_output = OutputCalculation(
                feds_firenrt,
                nifc_search,
                output_format, 
                output_maap_url,
                day_search_range,
                print_on,
                plot_on
                )

### Visualize/Print Results

As noted above, this plotting will be moved into the classes. For now, it is demonstrating the same functionality it will have in the future. Used returned indices printed out before to plot and visualize

In [ ]:
feds_kincade = feds_firenrt.polygons[feds_firenrt.polygons['index'] == 41] # <-- thats the index in (41, 11287)! 
feds_kincade.head()

In [ ]:
nifc_match = nifc_search.polygons[nifc_search.polygons['index'] == 11287] # <-- thats the index in (41, 11287)! 
nifc_match.head()

In [ ]:
# for debugging purposes this is here

import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(15, 15))

# bound box as defined by us
ax.set_xlim(-122.887312, -122.601345)
ax.set_ylim(38.560242, 38.813395)

# feds_kincade.t.values[0]

# plot search results/what calculation is pulling from
v = feds_kincade.plot(ax=ax, legend=True, label="FEDS Fire Estimate", color="blue",edgecolor="black", linewidth=0.5 )
n = nifc_match.plot(ax=ax, legend=True, label="NIFC Nearest Date + Intersection", color="gold", edgecolor="black", linewidth=0.5, alpha=0.7)


ax.set_title("FEDS in Kincade Region vs NIFC Polygon with closest date + touch")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
plt.show()
